# Marketing Growth Analysis - Mini Conversion Funnel & Performance Analysis

<h2> Marketing Campaign Performance Analysis 
<h3> ABC Company is into business for past 8 months and has carried out various Digitial Marketing Campaigns to boost the sales. Marketing team aims to present the performance metrics to the management. <br> The Data team is required to analyze the website traffic and performance data  from the Database to quantify the company's growth to tell the story of how the company has been able to generate the growth. 
<br> The marketing team also aims to analyse the customer journey characteristics to uncover opportunities to divert the marketing budget spending
</h3>


In [1]:
# importing Libraries
import os
import numpy as np
import pandas as pd
import sqlite3
from sqlite3 import Error

<h2> Lets get an Idea of how tables/files 'website_pageviews', 'website_sessions' & 'orders' look like

In [2]:
df = pd.read_csv('website_pageviews.csv',header=0)
df.head(3)

,website_pageview_id,created_at,website_session_id,pageview_url
0,1,2012-03-19 08:04:16,1,/home
1,2,2012-03-19 08:16:49,2,/home
2,3,2012-03-19 08:26:55,3,/home


In [3]:
website_sessions_df = pd.read_csv('website_sessions.csv',header=0)
website_sessions_df.head(3)

,website_session_id,created_at,user_id,is_repeat_session,utm_source,utm_campaign,utm_content,device_type,http_referer
0,1,2012-03-19 08:04:16,1,0,gsearch,nonbrand,g_ad_1,mobile,https://www.gsearch.com
1,2,2012-03-19 08:16:49,2,0,gsearch,nonbrand,g_ad_1,desktop,https://www.gsearch.com
2,3,2012-03-19 08:26:55,3,0,gsearch,nonbrand,g_ad_1,desktop,https://www.gsearch.com


In [4]:
orders_df = pd.read_csv('orders_maven.csv',header=0)
orders_df.head(3)

,order_id,created_at,website_session_id,user_id,primary_product_id,items_purchased,price_usd,cogs_usd
0,1,2012-03-19 10:42:46,20,20,1,1,49.99,19.49
1,2,2012-03-19 19:27:37,104,104,1,1,49.99,19.49
2,3,2012-03-20 06:44:45,147,147,1,1,49.99,19.49


## Create an SQL Lite In Memory DB & its Connection

In [5]:

import sqlite3
from sqlite3 import Error
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
    except Error as e:
        print(e)

    return conn

### Step 1 - Reading CSV Files into Data Frames and then writing into SQL Lite Tables (In-Memory)
### Step 2 - Reading from SQL Lite Tables, Performing necessary aggregations and displaying 

In [8]:
conn=create_connection(':memory:')
#website_sessions_df = spark.read.format("csv").option("header","true").option("inferSchema","true").load("website_sessions.csv")
website_pageviews_df = pd.read_csv('website_pageviews.csv',header=0)
website_pageviews_df.head()
website_sessions_df = pd.read_csv('website_sessions.csv',header=0)
website_sessions_df.head()
orders_df = pd.read_csv('orders_maven.csv',header=0)
#website_sessions_df = pd.read_csv('website_pageviews.csv')
cursor = conn.cursor()
tbl_name1= "website_pageviews"
cursor.execute("drop table if exists %s;"%(tbl_name1))
#print('Table {0} was created successfully'.format(tbl_name1))
website_pageviews_df.to_sql(tbl_name1,conn,if_exists='replace', index=False)
print('File %s copied to db'%(tbl_name1))
tbl_name2= "website_sessions"
cursor.execute("drop table if exists %s;"%(tbl_name2))
#print('Table {0} was created successfully'.format(tbl_name2))
website_sessions_df.to_sql(tbl_name2,conn,if_exists='replace', index=False)
print('File %s copied to db'%(tbl_name2))
# creating a third table [orders] in memory
tbl_name3= "orders"
cursor.execute("drop table if exists %s;"%(tbl_name3))
#print('Table {0} was created successfully'.format(tbl_name2))
orders_df.to_sql(tbl_name3,conn,if_exists='replace', index=False)
print('File %s copied to db'%(tbl_name3))
conn.commit()
#cursor=conn.cursor()
sql="select * from %s limit 10"%tbl_name1
sql_df1 = pd.read_sql('select * from website_sessions limit 3',con=conn)
sql_df2 = pd.read_sql('select * from website_pageviews limit 3',con=conn)
sql_df1.head(3)
#sql_df2.head(3)

File website_pageviews copied to db
File website_sessions copied to db
File orders copied to db


,website_session_id,created_at,user_id,is_repeat_session,utm_source,utm_campaign,utm_content,device_type,http_referer
0,1,2012-03-19 08:04:16,1,0,gsearch,nonbrand,g_ad_1,mobile,https://www.gsearch.com
1,2,2012-03-19 08:16:49,2,0,gsearch,nonbrand,g_ad_1,desktop,https://www.gsearch.com
2,3,2012-03-19 08:26:55,3,0,gsearch,nonbrand,g_ad_1,desktop,https://www.gsearch.com


<h3> Q # 01 - Marketing Team believes that Google Search (gsearch) seems to be the biggest driver of the Business, Kindly pull out the monthly trends for search sessions and orders so that we can showcase the relevant growth!

In [9]:
pd.read_sql("""
select strftime('%Y',website_sessions.created_at) yr,
strftime('%m',website_sessions.created_at) mo,
count(distinct website_sessions.website_session_id) sessions, 
 count(distinct orders.order_id) orders,
 round(count(distinct orders.order_id)*1.0/count(distinct website_sessions.website_session_id)*100,2) as conversion_rate
from website_sessions left join orders 
on website_sessions.website_session_id = orders.website_session_id
where utm_source='gsearch' 
and  website_sessions.created_at < '2012-11-27'
group by strftime('%Y',website_sessions.created_at),strftime('%m',website_sessions.created_at)
"""
,con=conn).head(10)

,yr,mo,sessions,orders,conversion_rate
0,2012,03,1860,60,3.23
1,2012,04,3574,92,2.57
2,2012,05,3410,97,2.84
3,2012,06,3578,121,3.38
4,2012,07,3811,145,3.80
5,2012,08,4877,184,3.77
6,2012,09,4491,188,4.19
7,2012,10,5534,234,4.23
8,2012,11,8889,373,4.20


<h3> RESULT - We can see that the Orders started growing in  number starting June Month

<h2> Q # 02
<h3> Now Marketing team would like to split the 'brand' and 'non-brand' campaigns separately for Google Search(gsearch)
<h3> Marketing team would like to know if campaign named 'brand' is picking up or not 
<h3> Brand (brand) here means that the customer is directly using our name in Google Search Engine

In [10]:
pd.read_sql("""
select strftime('%Y',website_sessions.created_at) yr,
	strftime('%m',website_sessions.created_at) yr_month,--utm_campaign,
	count(distinct  case when  utm_campaign = 'nonbrand' then website_sessions.website_session_id else null end ) nonbrand_sessions, 
	count(distinct case when  utm_campaign = 'nonbrand' then orders.order_id else null end ) nonbrand_orders,
	count(distinct  case when  utm_campaign = 'brand' then website_sessions.website_session_id else null end ) brand_sessions, 
 	count(distinct case when  utm_campaign = 'brand' then orders.order_id else null end ) brand_orders
from website_sessions left join orders 
on website_sessions.website_session_id = orders.website_session_id
where utm_source='gsearch' and utm_campaign in ('nonbrand', 'brand')
and  website_sessions.created_at < '2012-11-27' ---- 19-03-2012
group by strftime('%Y',website_sessions.created_at) ,
strftime('%m',website_sessions.created_at)
"""
,con=conn).head(10)

,yr,yr_month,nonbrand_sessions,nonbrand_orders,brand_sessions,brand_orders
0,2012,03,1852,60,8,0
1,2012,04,3509,86,65,6
2,2012,05,3295,91,115,6
3,2012,06,3439,114,139,7
4,2012,07,3660,136,151,9
5,2012,08,4673,174,204,10
6,2012,09,4227,172,264,16
7,2012,10,5197,219,337,15
8,2012,11,8506,356,383,17


</h3> Result, We can see 'brand' is gradually picking up  on a month-to-month basis, so customers are get awareness about our business 

<h2> Q # 03
<h3> Now we would like to dive into 'non-brand' category and pull the monthly sessions and orders split by device type i.e mobile, desktop 

In [11]:
pd.read_sql("""
select strftime('%Y',website_sessions.created_at) yr,
	strftime('%m',website_sessions.created_at)yr_mo,
	Count(distinct  case when device_type = 'desktop' then website_sessions.website_session_id else null end ) desktop_sessions, 
	count(distinct case when  device_type = 'desktop' then orders.order_id else null end ) desktop_orders,
	count(distinct  case when device_type = 'mobile' then website_sessions.website_session_id else null end ) mobile_sessions, 
 	count(distinct case when  device_type = 'mobile' then orders.order_id else null end ) mobile_orders
from website_sessions left join orders 
on website_sessions.website_session_id = orders.website_session_id
where utm_source='gsearch' and utm_campaign = 'nonbrand'
and  website_sessions.created_at < '2012-11-27' ---- 19-03-2012
group by strftime('%Y',website_sessions.created_at),
strftime('%m',website_sessions.created_at)
""" ,con=conn).head(10)

,yr,yr_mo,desktop_sessions,desktop_orders,mobile_sessions,mobile_orders
0,2012,03,1128,50,724,10
1,2012,04,2139,75,1370,11
2,2012,05,2276,83,1019,8
3,2012,06,2673,106,766,8
4,2012,07,2774,122,886,14
5,2012,08,3515,165,1158,9
6,2012,09,3171,155,1056,17
7,2012,10,3934,201,1263,18
8,2012,11,6457,323,2049,33


<h3> It can be seen that most of the orders are coming from Desktop and we witness overall growth from 3rd Month through 11 Month 

<h2> Q # 04
<h3> Although Google Search (gsearch ) is working fine, but one of the Marketing team member is concerned about the large %age from Google Search
<br> Lets pull a monthly trends for Google Search along with monthly trends for each of the other channels 

In [12]:
pd.read_sql("""
select 
	strftime('%Y',website_sessions.created_at) yr,
	strftime('%m',website_sessions.created_at) yr_month,
	count(distinct case when utm_source='gsearch' then  website_sessions.website_session_id else null end) 
	google_paid_sessions, 
	count(distinct case when utm_source='bsearch' then  website_sessions.website_session_id else null end) 
	bing_paid_sessions, 
	count(distinct case when utm_source is null and http_referer is not null then  website_sessions.website_session_id else null end) 
	organic_search_sessions, 
	count(distinct case when utm_source is null and http_referer is null then  website_sessions.website_session_id else null end) 
	direct_type_in_sessions
from website_sessions left join orders 
on website_sessions.website_session_id = orders.website_session_id
where -- utm_source='gsearch' --and utm_campaign = 'nonbrand' and 
website_sessions.created_at < '2012-11-27' ---- 19-03-2012
group by strftime('%Y',website_sessions.created_at), strftime('%m',website_sessions.created_at)
""" ,con=conn).head(10)

,yr,yr_month,google_paid_sessions,bing_paid_sessions,organic_search_sessions,direct_type_in_sessions
0,2012,03,1860,2,8,9
1,2012,04,3574,11,78,71
2,2012,05,3410,25,150,151
3,2012,06,3578,25,190,170
4,2012,07,3811,44,207,187
5,2012,08,4877,705,265,250
6,2012,09,4491,1439,331,285
7,2012,10,5534,1781,428,440
8,2012,11,8889,2840,536,485


<h3> We can see Google Search (gsearch) is still the largest contributor followed by Bing, Organic search and Direct Type In
<br> It can be noted very clearly that Organic search and Direct Type is showing steady growth

<h2> Q # 05
<h3> Marketing manager would like to know the website performance improvement over the course of first 8 months (March 2012 onwards)
<h3> Could you please pull session to order rates, month-wise

In [13]:
pd.read_sql("""
select strftime('%Y',created_at) yr,
strftime('%m',created_at) yr_month,
count( distinct website_session_id) sessions,
count(distinct order_id) as orders,
round(count(distinct order_id)*1.0 /count( distinct website_session_id)* 100, 3)conversion_rate
from (
	select website_sessions.website_session_id, 
	website_sessions.created_at, orders.order_id
	from website_sessions left join orders 
	on website_sessions.website_session_id = orders.website_session_id
	where utm_source='gsearch'
	and  website_sessions.created_at < '2012-11-27'
	 )x
group by strftime('%Y',created_at),strftime('%m',created_at)
"""
,con=conn).head(10)

,yr,yr_month,sessions,orders,conversion_rate
0,2012,03,1860,60,3.226
1,2012,04,3574,92,2.574
2,2012,05,3410,97,2.845
3,2012,06,3578,121,3.382
4,2012,07,3811,145,3.805
5,2012,08,4877,184,3.773
6,2012,09,4491,188,4.186
7,2012,10,5534,234,4.228
8,2012,11,8889,373,4.196


<h3> We Noticed that the Conversion Rate has been increasing starting June

<h2> Q # 06 
<h3> Marketing team would like to know If newly launched Landing Page (lander-1) has better appeal to the customer.
<br> Marketing team would like to know the revenue generated since the launch of Landing Page 'lander-1' while considering campaign 'nonbrand' & Google Search ('gsearch')
<br> Marketing team would like to see the comparison of new Landing Page viz-a-viz Old landing Page i.e 'home'
<h2> Steps
<h3> We will find out when new Landing Page ('lander-1') was created and first used 
<br> Take the sessions involving Page 'lander-1' & 'home' from that time onwards 
<br> Stitch together the data with the Orders which have been genearted from said sessions

In [14]:
pd.read_sql("""
With landers_sessions_q6 as
(
	select session_fpages_q6.website_session_id, 
	website_pageviews.pageview_url as landing_page
	from
	(
	select min(website_pageview_id) first_pageview_id,
	--website_pageview_id pageview_id --,pageview_url,
	website_pageviews.website_session_id
	from website_pageviews INNER JOIN website_sessions  
	on website_pageviews.website_session_id = website_sessions.website_session_id
	where website_pageviews.website_pageview_id >=(select min(website_pageview_id) from website_pageviews where pageview_url='/lander-1') 
	and website_pageviews.created_at < '2012-07-28' -- '2012-11-27'
	and utm_source='gsearch' and utm_campaign = 'nonbrand'
	group by website_pageviews.website_session_id
	) session_fpages_q6 left join website_pageviews
	on session_fpages_q6.first_pageview_id = website_pageviews.website_pageview_id
	where website_pageviews.pageview_url  in ('/home','/lander-1')
)
select landers_sessions_q6.landing_page,
count (landers_sessions_q6.website_session_id) as sessions,
count(distinct orders.order_id) as orders,
round(count(distinct orders.order_id) * 1.0 /
	count (landers_sessions_q6.website_session_id)*100 , 3)  as conversion_rate
from landers_sessions_q6 left join orders
on landers_sessions_q6.website_session_id = orders.website_session_id
group by landers_sessions_q6.landing_page
"""
,con=conn).head(10)

,landing_page,sessions,orders,conversion_rate
0,/home,2261,72,3.184
1,/lander-1,2316,94,4.059


<h3> Result: Landing Page named 'lander-1' is performing at 4.05% compared to 3.18%  performance rate of 'home' 
<br> We have got 0.87% additional orders

<h3> Step 3
<br>Marketing Team would like to know the number of sessions (starting from the time when the traffic was last sent to old landing page 'home') that has been re-routed excluding old Landing Page

In [15]:
pd.read_sql("""
select count (website_session_id) as Session_routed_since_test
from website_sessions
where created_at < '2012-11-27' and utm_source='gsearch' 
and utm_campaign = 'nonbrand' and website_session_id >
(
    select max (website_sessions.website_session_id) as most_recent_nonbrand_home_pageview
    from website_sessions
    left join website_pageviews
    on website_pageviews.website_session_id = website_sessions.website_session_id
    where website_pageviews.created_at < '2012-11-27'
    and utm_source='gsearch' and utm_campaign = 'nonbrand' and website_pageviews.pageview_url='/home'
)
"""
,con=conn).head()

,Session_routed_since_test
0,22972


<h3> Since the Test start date; a Total of 22,972 sessions has been re-routed 
<br> Using the Session to Order ratio it has been found that we have 2020 Incremental orders since Test Launch Date
<br> Over a period of four months we have got 50 extra orders per month.

- To be Continued


<h2> Q # 07
<h3> Marketing Team would like to know a full conversion funnel from each of the two pages to orders . [June-July 2012 period]

In [17]:
pd.read_sql("""
with sessions_level_counts_q7 as 
(
	select case when home_saw=1 then 'home_saw' when lander_saw=1 then 'lander_saw' else 'un-known' end as segment,
count( distinct website_session_id) as sessions,
count( distinct case when products_hit=1 then website_session_id else null end) products_hit,
count( distinct case when mrfuzzy_hit=1 then website_session_id else null end) mrfuzzy_hit,
count( distinct case when shipping_hit=1 then website_session_id else null end) shipping_hit,
count( distinct case when billing_hit=1 then website_session_id else null end) billing_hit,
count( distinct case when cart_hit=1 then website_session_id else null end) cart_hit, 
count( distinct case when thanks_hit=1 then website_session_id else null end )thanks_hit
from (
select 
website_session_id,
max(home_saw) home_saw,
max(lander_saw) lander_saw,
max(products_hit) products_hit,
max(mrfuzzy_hit) mrfuzzy_hit,
max(shipping_hit) shipping_hit,
max(billing_hit) billing_hit,
max(cart_hit)cart_hit,
max(thanks_hit) thanks_hit
	from
	(
	select
	website_sessions.website_session_id, --min(website_pageview_id) min_pageview --, 
	case when website_pageviews.pageview_url= '/home' then 1 else 0 end as home_saw,
	case when website_pageviews.pageview_url= '/lander-1' then 1 else 0 end as lander_saw,
	case when website_pageviews.pageview_url= '/products' then 1 else 0 end as products_hit,
	case when website_pageviews.pageview_url = '/the-original-mr-fuzzy' then 1 else 0 end as mrfuzzy_hit,
	case when website_pageviews.pageview_url= '/shipping' then 1 else 0 end as shipping_hit,
	case when website_pageviews.pageview_url ='/billing'  then 1 else 0 end as billing_hit,
	case when website_pageviews.pageview_url= '/cart' then 1 else 0 end as cart_hit,
	case when website_pageviews.pageview_url= '/thank-you-for-your-order' then 1 else 0 end as thanks_hit
	from website_pageviews inner join website_sessions
	on website_pageviews.website_session_id =  website_sessions.website_session_id
	where website_pageview_id  >= ( select min (website_pageview_id ) from website_pageviews where pageview_url='/lander-1')
	and website_pageviews.created_at <= '2012-07-28' -- '2012-06-19'
	and utm_source='gsearch' and utm_campaign ='nonbrand'
		)x
	group by website_session_id
	) sessions_pages_hit_q7
	group by 1 
)
select segment,
round(products_hit*1.0/sessions,3) as lander_cvr,
round(mrfuzzy_hit*1.0/products_hit,3) as products_cvr, 
round(cart_hit*1.0/mrfuzzy_hit,3) as mrfuzzy_cvr,
round(shipping_hit*1.0/ cart_hit,3) as cart_cvr,
round(billing_hit*1.0/shipping_hit,3) as shipping_cvr, 
round(thanks_hit*1.0/billing_hit,3) as thanks_cvr
from sessions_level_counts_q7
"""
,con=conn).head()

,segment,lander_cvr,products_cvr,mrfuzzy_cvr,cart_cvr,shipping_cvr,thanks_cvr
0,home_saw,0.417,0.726,0.433,0.676,0.840,0.429
1,lander_saw,0.468,0.713,0.451,0.664,0.853,0.477


<h3> Result: It can be seen that new Landing page has shown better conversion rate than home landing page for period June-July 2012

<h2> Q # 8
<h3> Finally we would like to Quantify the Impact of a billing Test (A new Billing Page was for easy user experience)
<br> Kindly Analyse the lift generated from the Billing Page Test in terms of Revenue per billing page session 
<br> Then get the number of billing page sessions for the past month to understand the monthly impact

In [18]:
pd.read_sql("""
select website_pageviews.pageview_url as landing_page,
count(distinct website_pageviews.website_session_id) sessions, 
sum(orders.price_usd) revenue,
ROUND(sum(orders.price_usd)*1.0/count(distinct website_pageviews.website_session_id),3) as revenue_session
from website_pageviews left join orders
on website_pageviews.website_session_id =  orders.website_session_id
where website_pageviews.created_at > '2012-09-10' and website_pageviews.created_at < '2012-11-10' 
and website_pageviews.pageview_url in ('/billing','/billing-2')
group by 1
"""
,con=conn).head()

,landing_page,sessions,revenue,revenue_session
0,/billing,657,14997.0,22.826
1,/billing-2,654,20495.9,31.339


<h3> Results - New version brings in revenue of $31.33 vs $ 22.82 of old page
<br> An increase of $8.5 per billing page view
<br> <br>
Next Step - Lets Find the session count where new billing page was used in PAST ONE MONTH date range

In [19]:
pd.read_sql("""
select count ( distinct website_session_id) session_count
from website_pageviews
where created_at > '2012-10-27' and created_at < '2012-11-27' 
and pageview_url in ('/billing','/billing-2')
"""
,con=conn).head()

,session_count
0,1193


<h3> We have a total of 1193 sessions for the Billing Page
<br> So we have Incremental revenue of $ 10,140 (1311* 8.50 = $ 10,140) over past one month till ...  27 Nov-2012

## Q # 09 :  Conversion Funnel Analysis - For a Target Product 
* Its about understanding and Optimizing each step of the User Experience on their Journey towards purchase of our product 
* Use Case 1- Identifying the most common Paths customers take before purchasing our product 
* Use Case 2- Identifying how many of our users continue on to next step in conversion flow & how mnay abandon at each step
* Use Case 3-  Optimizing critical pain points where users are abandoning so that we can convert more users & sell more products 
    * Example 95 users  visited home page, 65 from among those 95 visited products page, 45 from among those 65 visited  to add to cart page for final purchase
    * So 65/95 gives us the Click through rate CTR for Products Page and so on ...
### Lets build a mini conversion funnel from a Landing Page ('lander-1') to Shopping Cart ('/cart')
### Lets see how many people reach each step in their Customer Journey, and drop off  rates
### We are focussing at customer who are looking at a specific product 

In [20]:
session_level_made_it_flag_demo = pd.read_sql("""
select website_session_id, 
max(products_page) products_made_it,
max(mrfuzzy_page) mrfuzzy_made_it,
max(cart_page) cart_page_made_it
from 
(
select website_sessions.website_session_id, 
website_pageviews.pageview_url, 
website_pageviews.created_at pageview_created_at,
case when website_pageviews.pageview_url= '/products' then 1 else 0 end as products_page,
case when website_pageviews.pageview_url= '/the-original-mr-fuzzy' then 1 else 0 end as mrfuzzy_page,
case when website_pageviews.pageview_url= '/cart' then 1 else 0 end as cart_page
from website_sessions left join website_pageviews
on website_sessions.website_session_id = website_pageviews.website_session_id
where 
	 --website_pageviews.pageview_url = (select min(pageview_url) from website_pageviews group by website_session_id)
	--website_sessions.created_at>='2012-03-19 08:26:55' and 
website_pageviews.pageview_url in ('/the-original-mr-fuzzy','/lander-2','/products','/cart') 
order by website_sessions.website_session_id
)x	
group by website_session_id
order by website_session_id
"""
,con=conn)
tbl_name3= "session_level_made_it_flag_demo"
session_level_made_it_flag_demo.to_sql(tbl_name3,conn,if_exists='replace', index=False)

In [21]:
pd.read_sql("""
select count(distinct website_session_id) as sessions,
round(count(distinct case when products_made_it=1 then website_session_id else null end )*1.0/
count(distinct website_session_id),3) as lander_click_through_rate , 
round(count(distinct case when mrfuzzy_made_it =1 then website_session_id else null end )*1.0/
count(distinct website_session_id),3) as products_through_rate,
ROUND(count(distinct case when cart_page_made_it =1 then website_session_id else null end )*1.0/
count(distinct website_session_id),3) as target_product_click_through_rate
from session_level_made_it_flag_demo
"""
,con=conn).head()

,sessions,lander_click_through_rate,products_through_rate,target_product_click_through_rate
0,320480,0.815,0.507,0.296


<h3> Result : We can see that the Target Product has a click through rate of 29.6%

<h1> Thank you for Reading